In [1]:
# importando a biblioteca para fazer operações em banco de dados sqlite3
import sqlite3
# importando a bibliotecas pandas para transformar dados e tabelas do banco em data frames 
import pandas as pd

In [3]:
# criaremos a conexão com nosso banco de dados
# se ele não existir, será criado
conn = sqlite3.connect("database.db")

In [5]:
# Podemos agora criar tabelas
# criaremos a tabela (table) tasks com campos (fields) id, description e done
conn.execute("""
    CREATE TABLE IF NOT EXISTS tasks
    (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        description TEXT,
        done INTEGER
    )    
""")
# após executar uma operação é necessário salvá-la
conn.commit()

In [7]:
# criaremos agora um registro (row) na tabela tasks
# o registro terá valor 'Estudar seaborn' para description e 0 para done.
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar seaborn', 0);    
""")
# novamente, após executar o comando é necessário salvar
conn.commit()

In [9]:
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Estudar python', 1)
""")

# Salvar as alterações
conn.commit()


In [11]:
conn.execute("""
    INSERT INTO tasks (description, done)
    VALUES ('Entregar livro', 0)
""")

# Salvar as alterações
conn.commit()


In [13]:
# podemos agora visualizar todos (*) os registros da (FROM) tabela tasks
results = conn.execute("""
    SELECT *
    FROM tasks
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

(1, 'Estudar seaborn', 0)
(2, 'Estudar python', 1)
(3, 'Entregar livro', 0)


In [15]:
# ver os valores de description da (FROM) tabela tasks onde (WHERE) done = 0
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done=0
""").fetchall()
# imprimindo cada resultado
for result in results:
    print(result)

('Estudar seaborn',)
('Entregar livro',)


In [17]:
results = conn.execute("""
    SELECT description
    FROM tasks
    WHERE done = 0 AND description LIKE 'Estudar%'
""").fetchall()

# imprimindo cada resultado
for result in results:
    print(result)


('Estudar seaborn',)


In [19]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo id = 1 fazendo (SET) done = 1
conn.execute("""
    UPDATE tasks
    SET done=1
    WHERE id=1
""")
# salvando as atualizações
conn.commit()

In [21]:
conn.execute("""
    UPDATE tasks
    SET description = 'Estudar matplotlib'
    WHERE description = 'Estudar seaborn'
""")

# salvando as atualizações
conn.commit()


In [23]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn.execute("""
    DELETE FROM tasks
    WHERE id=1
""")
conn.commit()

In [25]:
df = pd.read_sql_query(
    """
       SELECT *
       FROM tasks
    """,
    conn
)
df

,id,description,done
0,2,Estudar python,1
1,3,Entregar livro,0


In [27]:
df['done'].value_counts()

done
1    1
0    1
Name: count, dtype: int64

In [29]:
results = conn.execute("""
    SELECT done, COUNT(*) AS done_tasks
    FROM tasks
    GROUP BY done;      
""").fetchall()

for result in results:
    print(result)
conn.close()

(0, 1)
(1, 1)


In [31]:
conn = sqlite3.connect("database_2.db")

conn.execute("""
    CREATE TABLE categories 
    (
      id    INTEGER PRIMARY KEY, 
      name  TEXT NOT NULL
    );
""")

conn.execute(
    """
        INSERT INTO categories (name)
        VALUES
          ('casa'),
          ('laser'),
          ('faculdade')
    """
)
conn.commit()

In [33]:
conn.execute("""
    CREATE TABLE tasks 
    (
      id            INTEGER PRIMARY KEY AUTOINCREMENT, 
      description   TEXT NOT NULL,
      done          INTEGER,
      category_id   INTEGER NOT NULL,
      FOREIGN KEY (category_id) REFERENCES categories (id)
    )
""")
conn.commit()

In [35]:
# insert many
conn.execute(
    """
    INSERT INTO 'tasks' (description, done, category_id)
    VALUES
      ('Assistir Netflix', 1, 2),
      ('Estudar matplotlib', 0, 3),
      ('Estudar seaborn', 0, 3),
      ('Estudar Pandas', 0, 3),
      ('Lavar louça', 0, 1)
    """
)
conn.commit()

In [37]:
tasks_with_categories = conn.execute(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """
).fetchall()

for task in tasks_with_categories:
    print(task)

(5, 'Lavar louça', 0, 'casa')
(2, 'Estudar matplotlib', 0, 'faculdade')
(3, 'Estudar seaborn', 0, 'faculdade')
(4, 'Estudar Pandas', 0, 'faculdade')
(1, 'Assistir Netflix', 1, 'laser')


In [39]:
df = pd.read_sql_query(
    """
    SELECT t.id, t.description, t.done, c.name 
    FROM tasks AS t 
    INNER JOIN categories AS c
      ON c.id = t.category_id
    ORDER BY c.name
    """,
    conn
)
df


,id,description,done,name
0,5,Lavar louça,0,casa
1,2,Estudar matplotlib,0,faculdade
2,3,Estudar seaborn,0,faculdade
3,4,Estudar Pandas,0,faculdade
4,1,Assistir Netflix,1,laser


In [41]:
conn.close()

In [45]:
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
        SELECT 
    CustomerId, 
    FirstName || ' ' || LastName AS NomeCompleto, 
    Country AS País
FROM customers
WHERE Country != 'USA'
ORDER BY Country ASC, NomeCompleto ASC;

    """,
    conn
)
conn.close()
df

,CustomerId,NomeCompleto,País
0,56,Diego Gutiérrez,Argentina
1,55,Mark Taylor,Australia
2,7,Astrid Gruber,Austria
3,8,Daan Peeters,Belgium
4,11,Alexandre Rocha,Brazil
5,10,Eduardo Martins,Brazil
6,13,Fernanda Ramos,Brazil
7,1,Luís Gonçalves,Brazil
8,12,Roberto Almeida,Brazil
9,32,Aaron Mitchell,Canada


In [51]:
#Fornecer uma consulta que mostre apenas os clientes do Brasil.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT FirstName || ' ' || LastName AS Nome, City AS Cidade, State AS Estado, Country AS País
    FROM customers
    WHERE Country = 'Brazil'
    ORDER BY Nome ASC
    """,
    conn
)
conn.close()
df

,Nome,Cidade,Estado,País
0,Alexandre Rocha,São Paulo,SP,Brazil
1,Eduardo Martins,São Paulo,SP,Brazil
2,Fernanda Ramos,Brasília,DF,Brazil
3,Luís Gonçalves,São José dos Campos,SP,Brazil
4,Roberto Almeida,Rio de Janeiro,RJ,Brazil


In [53]:
#Fornecer uma consulta que mostre as faturas dos clientes que são do Brasil. 
#A tabela resultante deve mostrar o nome completo do cliente, ID da fatura, data da fatura e país de cobrança.

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        c.FirstName || ' ' || c.LastName AS Nome,
        i.InvoiceId AS IDFatura,
        i.InvoiceDate AS DataFatura,
        i.BillingCountry AS País
    FROM customers c
    JOIN invoices i ON c.CustomerId = i.CustomerId
    WHERE c.Country = 'Brazil'
    ORDER BY Nome ASC

    """,
    conn
)
conn.close()
df

,Nome,IDFatura,DataFatura,País
0,Alexandre Rocha,57,2009-09-06 00:00:00,Brazil
1,Alexandre Rocha,68,2009-10-17 00:00:00,Brazil
2,Alexandre Rocha,123,2010-06-17 00:00:00,Brazil
3,Alexandre Rocha,252,2012-01-22 00:00:00,Brazil
4,Alexandre Rocha,275,2012-04-25 00:00:00,Brazil
5,Alexandre Rocha,297,2012-07-28 00:00:00,Brazil
6,Alexandre Rocha,349,2013-03-18 00:00:00,Brazil
7,Eduardo Martins,25,2009-04-09 00:00:00,Brazil
8,Eduardo Martins,154,2010-11-14 00:00:00,Brazil
9,Eduardo Martins,177,2011-02-16 00:00:00,Brazil


In [63]:
#Fornecer uma consulta que mostre apenas os funcionários que são agentes de vendas.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT EmployeeId, FirstName || ' ' || LastName AS Nome, Title AS 'Agentes de vendas'
    FROM employees
    WHERE Title = 'Sales Support Agent'
    ORDER BY Nome ASC

    """,
    conn
)
conn.close()
df

,EmployeeId,Nome,Agentes de vendas
0,3,Jane Peacock,Sales Support Agent
1,4,Margaret Park,Sales Support Agent
2,5,Steve Johnson,Sales Support Agent


In [65]:
#Fornecer uma consulta que mostre uma lista única de países de cobrança da tabela de faturas.
#DISTINCT = "Mostre apenas valores únicos (sem repetições)"
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT DISTINCT BillingCountry AS 'Países de cobrança'
    FROM Invoices
    ORDER BY BillingCountry ASC

    """,
    conn
)
conn.close()
df

,Países de cobrança
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Canada
6,Chile
7,Czech Republic
8,Denmark
9,Finland


In [69]:
#Fornecer uma consulta que mostre as faturas associadas a cada agente de vendas. 
#A tabela resultante deve incluir o nome completo do agente de vendas.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        e.FirstName || ' ' || e.LastName AS Nome,
        i.InvoiceId,
        i.InvoiceDate AS Data,
        i.BillingCountry AS 'Países de Cobrança'
    FROM Invoices i
    JOIN Customers c ON i.CustomerId = c.CustomerId
    JOIN Employees e ON c.SupportRepId = e.EmployeeId
    WHERE e.Title = 'Sales Support Agent'
    ORDER BY Nome, i.InvoiceId

    """,
    conn
)
conn.close()
df

,Nome,InvoiceId,Data,Países de Cobrança
0,Jane Peacock,6,2009-01-19 00:00:00,Germany
1,Jane Peacock,7,2009-02-01 00:00:00,Germany
2,Jane Peacock,9,2009-02-02 00:00:00,France
3,Jane Peacock,10,2009-02-03 00:00:00,Ireland
4,Jane Peacock,11,2009-02-06 00:00:00,United Kingdom
...,...,...,...,...
407,Steve Johnson,398,2013-10-21 00:00:00,France
408,Steve Johnson,402,2013-11-05 00:00:00,Spain
409,Steve Johnson,404,2013-11-13 00:00:00,Czech Republic
410,Steve Johnson,406,2013-12-04 00:00:00,USA


In [73]:
#É possível então saber quantas vendas e qual valor cada agente fez. Como?
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT
    e.FirstName || ' ' || e.LastName AS Nome,
    COUNT(i.InvoiceId) AS 'Numero Vendas',
    SUM(i.Total) AS 'Valor Total de Vendas'
    FROM Invoices i
    JOIN Customers c ON i.CustomerId = c.CustomerId
    JOIN Employees e ON c.SupportRepId = e.EmployeeId
    GROUP BY Nome
    ORDER BY 'Valor Total de Vendas' DESC;

    """,
    conn
)
conn.close()
df

,Nome,Numero Vendas,Valor Total de Vendas
0,Steve Johnson,126,720.16
1,Margaret Park,140,775.40
2,Jane Peacock,146,833.04


In [77]:
#Fornecer uma consulta que mostre o valor total da fatura, nome do cliente, país e nome do agente de vendas de todas as faturas e clientes.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT
        i.Total AS 'Valor Total Fatura',
        c.FirstName || ' ' || c.LastName AS 'Nome Cliente',
        c.Country AS País,
        e.FirstName || ' ' || e.LastName AS 'Nome Agente'
    FROM Invoices i
    JOIN Customers c ON i.CustomerId = c.CustomerId
    JOIN Employees e ON c.SupportRepId = e.EmployeeId

    """,
    conn
)
conn.close()
df

,Valor Total Fatura,Nome Cliente,País,Nome Agente
0,3.98,Luís Gonçalves,Brazil,Jane Peacock
1,3.96,Luís Gonçalves,Brazil,Jane Peacock
2,5.94,Luís Gonçalves,Brazil,Jane Peacock
3,0.99,Luís Gonçalves,Brazil,Jane Peacock
4,1.98,Luís Gonçalves,Brazil,Jane Peacock
...,...,...,...,...
407,5.94,Puja Srivastava,India,Jane Peacock
408,1.99,Puja Srivastava,India,Jane Peacock
409,1.98,Puja Srivastava,India,Jane Peacock
410,13.86,Puja Srivastava,India,Jane Peacock


In [83]:
#É possível agora saber qual nosso melhor cliente por valor ou quantidade vendida. Como?
#Cliente com maior valor total de compras

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
        SELECT
        c.FirstName || ' ' || c.LastName AS NomeCliente,
        COUNT(i.InvoiceId) AS NumeroVendas,
        SUM(i.Total) AS ValorTotalCompras
    FROM Invoices i
    JOIN Customers c ON i.CustomerId = c.CustomerId
    GROUP BY NomeCliente
    ORDER BY ValorTotalCompras DESC
    LIMIT 1;

    """,
    conn
)
conn.close()
df

,NomeCliente,NumeroVendas,ValorTotalCompras
0,Helena Holý,7,49.62


In [85]:
#Cliente com maior quantidade de vendas (faturas)

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
        SELECT
            c.FirstName || ' ' || c.LastName AS NomeCliente,
            COUNT(i.InvoiceId) AS NumeroVendas,
            SUM(i.Total) AS ValorTotalCompras
        FROM Invoices i
        JOIN Customers c ON i.CustomerId = c.CustomerId
        GROUP BY NomeCliente
        ORDER BY NumeroVendas DESC
        LIMIT 1;

    """,
    conn
)
conn.close()
df

,NomeCliente,NumeroVendas,ValorTotalCompras
0,Wyatt Girard,7,39.62


In [89]:
#Quantas faturas houve em 2009 e 2011? Quais são as vendas totais respectivas para cada um desses anos?
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        STRFTIME('%Y', InvoiceDate) AS Ano,
        COUNT(InvoiceId) AS 'Quantidade Faturas',
        SUM(Total) AS 'Vendas Totais'
    FROM Invoices
    WHERE Ano IN ('2009', '2011')
    GROUP BY Ano
    ORDER BY Ano;
    """,
    conn
)
conn.close()
df

,Ano,Quantidade Faturas,Vendas Totais
0,2009,83,449.46
1,2011,83,469.58


In [93]:
#Analisando a tabela invoices, forneça uma consulta que CONTE o número de itens para o ID da fatura 37.

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT InvoiceId, COUNT(*) AS 'Numero Itens'
    FROM Invoice_items
    WHERE InvoiceId = 37
    GROUP BY InvoiceId
    """,
    conn
)
conn.close()
df


,InvoiceId,Numero Itens
0,37,4


In [97]:
#Analisando a tabela invoice_items, forneça uma consulta que CONTE o número de itens para cada fatura. 
#DICA: AGRUPAR POR
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT InvoiceId, COUNT(*) AS 'Numero Itens'
    FROM Invoice_items
    GROUP BY InvoiceId
    ORDER BY 'Numero Itens' DESC
    """,
    conn
)
conn.close()
df

,InvoiceId,Numero Itens
0,1,2
1,2,4
2,3,6
3,4,9
4,5,14
...,...,...
407,408,4
408,409,6
409,410,9
410,411,14


In [99]:
#Fornecer uma consulta que inclua o nome da faixa para cada linha da tabela de itens da fatura.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT il.InvoiceLineId,
           il.InvoiceId,
           il.TrackId,
           t.Name AS TrackName
    FROM Invoice_Items il
    JOIN Tracks t ON il.TrackId = t.TrackId
    """,
    conn
)
conn.close()
df

,InvoiceLineId,InvoiceId,TrackId,TrackName
0,1,1,2,Balls to the Wall
1,2,1,4,Restless and Wild
2,3,2,6,Put The Finger On You
3,4,2,8,Inject The Venom
4,5,2,10,Evil Walks
...,...,...,...,...
2235,2236,411,3136,Looking For Love
2236,2237,411,3145,Sweet Lady Luck
2237,2238,411,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...
2238,2239,411,3163,Samba pras moças


In [101]:
#Forneça uma consulta que mostre o nome da música e o artista para cada linha da tabela de itens da fatura.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """  
    SELECT 
        il.InvoiceLineId,
        t.Name AS TrackName,
        ar.Name AS ArtistName
    FROM Invoice_Items il
    JOIN Tracks t ON il.TrackId = t.TrackId
    JOIN Albums al ON t.AlbumId = al.AlbumId
    JOIN Artists ar ON al.ArtistId = ar.ArtistId

    """,
    conn
)
conn.close()
df

,InvoiceLineId,TrackName,ArtistName
0,579,For Those About To Rock (We Salute You),AC/DC
1,1,Balls to the Wall,Accept
2,1154,Balls to the Wall,Accept
3,1728,Fast As a Shark,Accept
4,2,Restless and Wild,Accept
...,...,...,...
2235,1726,"Symphony No. 2, Op. 16 - ""The Four Temperamen...",Göteborgs Symfoniker & Neeme Järvi
2236,577,"Étude 1, In C Major - Preludio (Presto) - Liszt",Michele Campanella
2237,1153,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Eugene Ormandy
2238,578,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Emerson String Quartet


In [103]:
#Fornecer uma consulta que mostre o número de faturas por país. DICA: AGRUPAR POR
# TODO
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
    BillingCountry, 
    COUNT(InvoiceId) AS NumeroDeFaturas
    FROM Invoices
    GROUP BY BillingCountry
    ORDER BY NumeroDeFaturas DESC

    """,
    conn
)
conn.close()
df

,BillingCountry,NumeroDeFaturas
0,USA,91
1,Canada,56
2,France,35
3,Brazil,35
4,Germany,28
5,United Kingdom,21
6,Portugal,14
7,Czech Republic,14
8,India,13
9,Sweden,7


In [105]:
#Forneça uma consulta que mostra o número total de música por playlist. O nome e ID das playlists devem aparecer na tabela resultante.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """    
    SELECT 
        p.PlaylistId, 
        p.Name AS NomePlaylist, 
        COUNT(pt.TrackId) AS NumeroDeMusicas
    FROM playlists p
    JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId
    GROUP BY p.PlaylistId, p.Name
    ORDER BY NumeroDeMusicas DESC
    """,
    conn
)
conn.close()
df

,PlaylistId,NomePlaylist,NumeroDeMusicas
0,1,Music,3290
1,8,Music,3290
2,5,90’s Music,1477
3,3,TV Shows,213
4,10,TV Shows,213
5,12,Classical,75
6,11,Brazilian Music,39
7,17,Heavy Metal Classic,26
8,13,Classical 101 - Deep Cuts,25
9,14,Classical 101 - Next Steps,25


In [107]:
#Forneça uma consulta que mostra todos os nomes das músicas, mas não mostra ids. 
#A tabela resultante deve incluir o nome do álbum, tipo de media e gênero.

conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """    
    SELECT 
        t.Name AS NomeDaMusica,
        al.Title AS NomeDoAlbum,
        mt.Name AS TipoMidia,
        g.Name AS Genero
    FROM tracks t
    JOIN albums al ON t.AlbumId = al.AlbumId
    JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId
    JOIN genres g ON t.GenreId = g.GenreId
    ORDER BY t.Name
    """,
    conn
)
conn.close()
df

,NomeDaMusica,NomeDoAlbum,TipoMidia,Genero
0,"""40""",War,MPEG audio file,Rock
1,"""?""","Lost, Season 2",Protected MPEG-4 video file,TV Shows
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Sir Neville Marriner: A Celebration,Protected AAC audio file,Classical
3,#1 Zero,Out Of Exile,MPEG audio file,Alternative & Punk
4,#9 Dream,Instant Karma: The Amnesty International Campa...,Protected AAC audio file,Pop
...,...,...,...,...
3498,É que Nessa Encarnação Eu Nasci Manga,Axé Bahia 2001,MPEG audio file,Pop
3499,"Étude 1, In C Major - Preludio (Presto) - Liszt",Liszt - 12 Études D'Execution Transcendante,Purchased AAC audio file,Classical
3500,Óculos,Arquivo Os Paralamas Do Sucesso,MPEG audio file,Latin
3501,Óia Eu Aqui De Novo,As Canções de Eu Tu Eles,MPEG audio file,Soundtrack


In [109]:
#Fornecer uma consulta que mostra o número total de vendas de cada vendedor.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        e.FirstName || ' ' || e.LastName AS NomeVendedor,
        COUNT(i.InvoiceId) AS TotalDeVendas
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    JOIN invoices i ON c.CustomerId = i.CustomerId
    WHERE e.Title = 'Sales Support Agent'
    GROUP BY e.EmployeeId
    ORDER BY TotalDeVendas DESC

    """,
    conn
)
conn.close()
df

,NomeVendedor,TotalDeVendas
0,Jane Peacock,146
1,Margaret Park,140
2,Steve Johnson,126


In [111]:
#Qual vendedor (sales agent) mais vendeu em 2009?
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        e.FirstName || ' ' || e.LastName AS NomeVendedor,
        SUM(i.Total) AS TotalVendas
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    JOIN invoices i ON c.CustomerId = i.CustomerId
    WHERE e.Title = 'Sales Support Agent'
      AND strftime('%Y', i.InvoiceDate) = '2009'
    GROUP BY e.EmployeeId
    ORDER BY TotalVendas DESC
    LIMIT 1

    """,
    conn
)
conn.close()
df

,NomeVendedor,TotalVendas
0,Steve Johnson,164.34


In [113]:
#Qual vendedor (sales agent) mais vendeu considerando toda base de dados?
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        e.FirstName || ' ' || e.LastName AS NomeVendedor,
        SUM(i.Total) AS TotalVendas
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    JOIN invoices i ON c.CustomerId = i.CustomerId
    WHERE e.Title = 'Sales Support Agent'
    GROUP BY e.EmployeeId
    ORDER BY TotalVendas DESC
    LIMIT 1

    """,
    conn
)
conn.close()
df

,NomeVendedor,TotalVendas
0,Jane Peacock,833.04


In [115]:
#Forneça uma consulta que mostra o número de clientes inscritos para cada vendedor (sales agente).
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        e.FirstName || ' ' || e.LastName AS NomeVendedor,
        COUNT(c.CustomerId) AS NumeroDeClientes
    FROM employees e
    JOIN customers c ON e.EmployeeId = c.SupportRepId
    WHERE e.Title = 'Sales Support Agent'
    GROUP BY e.EmployeeId
    ORDER BY NumeroDeClientes DESC

    """,
    conn
)
conn.close()
df

,NomeVendedor,NumeroDeClientes
0,Jane Peacock,21
1,Margaret Park,20
2,Steve Johnson,18


In [117]:
#Forneça uma consulta que mostra o total de vendas por país.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT 
        BillingCountry AS Pais,
        SUM(Total) AS TotalVendas
    FROM invoices
    GROUP BY BillingCountry
    ORDER BY TotalVendas DESC

    """,
    conn
)
conn.close()
df

,Pais,TotalVendas
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


In [119]:
#Forneça uma consulta que mostra a faixa mais comprada de 2013.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT
        t.Name AS Faixa,
        COUNT(ii.TrackId) AS QuantidadeComprada
    FROM invoice_items ii
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    JOIN tracks t ON ii.TrackId = t.TrackId
    WHERE strftime('%Y', i.InvoiceDate) = '2013'
    GROUP BY ii.TrackId
    ORDER BY QuantidadeComprada DESC
    LIMIT 1

    """,
    conn
)
conn.close()
df

,Faixa,QuantidadeComprada
0,Black Diamond,1


In [121]:
#Forneça uma consulta que mostra os top 3 artistas mais vendidos de toda base de dados.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT ar.Name AS Artist, COUNT(ii.InvoiceLineId) AS TotalSales
    FROM artists ar
    JOIN albums al ON ar.ArtistId = al.ArtistId
    JOIN tracks t ON al.AlbumId = t.AlbumId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY ar.ArtistId
    ORDER BY TotalSales DESC
    LIMIT 3;

    """,
    conn
)
conn.close()
df

,Artist,TotalSales
0,Iron Maiden,140
1,U2,107
2,Metallica,91


In [ ]:
#Forneça uma query que mostra qual o tipo de media mais comprado.
conn = sqlite3.connect("assets/chinook.db")
df = pd.read_sql_query(
    """
    SELECT mt.Name AS MediaType, COUNT(ii.InvoiceLineId) AS TotalSales
    FROM media_types mt
    JOIN tracks t ON mt.MediaTypeId = t.MediaTypeId
    JOIN invoice_items ii ON t.TrackId = ii.TrackId
    GROUP BY mt.MediaTypeId
    ORDER BY TotalSales DESC
    LIMIT 1;

    """,
    conn
)
conn.close()
df